In [ ]:
#@title **MONTAR EL DRIVE** { display-mode: "form" }
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/ia2_project/Tweets Generator')
print(os.getcwd())
print(os.listdir())

Mounted at /content/drive/
/content/drive/My Drive/ia2_project/Trabajo Christian
['cached_lm_GPT2TokenizerFast_128_train_dataset.txt', 'cached_lm_GPT2TokenizerFast_128_test_dataset.txt', 'gpt2-tuits', 'runs', 'gpt2-tuits+gobierno', 'gpt2-gobierno', 'gpt2-gobierno_5', 'gpt2-gobierno_10', 'gpt2-tuits+gobierno_40', 'gpt2-tuits+gobierno_50', 'test_dataset.txt', 'train_dataset.txt', 'cached_lm_GPT2TokenizerFast_128_train_dataset.txt.lock', 'cached_lm_GPT2TokenizerFast_128_test_dataset.txt.lock', 'Text_Generator.ipynb']


# **Modelos de Generación de Texto - Estado del Arte**


---



Dentro de los modelos de generación de texto, quizás haya escuchado alguna vez del modelo de lenguaje GPT-2, o más recientemente GPT-3 de OpenAI. En donde estos modelos son capaces hasta de escribir código, como JSX, o HTML.

Una desventaja del modelo GPT-3 son sus 175 mil millones de parámetros (175B), lo que da como resultado un tamaño de modelo de alrededor de 350 GB. A modo de comparación, el modelo más grande de GPT-2 tiene 1,5 mil millones de parámetros. Esto es, menos de 1/116 del tamaño de GPT-3.


De hecho, con cerca de 175B de parámetros entrenables, GPT-3 es mucho más grande en términos de tamaño en comparación con cualquier otro modelo que existe en la actualidad. A continuación se puede ver una comparación de la cantidad de parámetros de los modelos populares recientes de NLP, GPT-3 se destaca claramente.

<img src="https://miro.medium.com/max/2400/1*jBl-cX-CmFliPxR3AabBFA.png" /></

Ahora bien, para entrenar un modelo de estas magnitudes, es necesario tanto de una cantidad de datos demasiado enorme, así como de un poder de computo abismal. 

Como dato curioso, entrenar GPT-3 tomaría alrededor de **355 años** en una Tesla V100, la GPU más rápida del mercado. Y costaría un estimado de $4,600,000 para ser entrenada con el proveedor de GPU más barato en la nube.

<center><img src="https://images.contentstack.io/v3/assets/blt71da4c740e00faaa/blt5f77997b6858e850/60149e0f0069f70f7772189f/Approximate-size-comparison-of-GPT-2.jpg" /></center>

Por otra parte, para hacernos una idea de la cantidad de datos requerida para entrenar estos modelos, se conoce que, para entrenar el modelo GPT-2 de 1.5B de parametros, se utilizó un dataset de 8 millones de páginas web. Por todo esto, y debido a que para efectos de la realización de nuestro proyecto, sería imposible entrenar modelos de estas magnitudes; incursionaremos en el modelo GPT-2, el cual cuenta con modelos más pequeños (con menos parámetros) pero que hacen posible llevar a cabo un proceso de fine-tune.

## **Modelos GPT-2 - Número de Parámetros**



<img src="https://jalammar.github.io/images/gpt2/gpt2-sizes.png" />

Debido a que estos modelos han sido entrenados en Inglés, nuestra tarea de implementar un modelo de generación de texto en Español adquiere mayor dificultad. Pensar en re-entrenar el modelo desde cero sería una tarea demasiado costosa en términos de información, poder computacional y dinero. Por lo cual, es más realista conseguir información o datasets menos robustos y hacer fine-tune a un modelo pre-entrenado en Inglés.





---



### **¿Qué es hacer Fine-Tune?**

Fine-tune significa tomar pesos de una red neuronal ya entrenada y usarla como inicialización para un nuevo modelo que se va a entrenar con datos del mismo dominio. Es decir, para nuestro caso particular, la re-utilización del vocabulario y las matrices de embeddings (todos los vectores de tokens **en común** entre Inglés y Español se mantienen), así como los pesos del modelo aprendidos en un corpus en Inglés (40gb de texto).

Esta estrategia es factible porque las reglas gramáticales entre el Inglés y Español, pese a ser diferentes, guardan algo de similitud.



---




## **Arquitectura GPT-2**


El modelo GPT-2 es un modelo emblemático de la revolución de los transformers en NLP desde 2017. Se ha caracterizado por ser capaz de escribir textos de una calidad cercana al nivel de los humanos. Pese a que actualmente ha sido superado en cuestión de parámetros y desempeño por modelos como BART, T5 y GPT-3, sigue siendo una referencia y objeto de estudio e investigación en la actualidad.

Este modelo se basa en la parte **decoder** de la arquitectura Transformer.

## **¿Cómo funcionan los Transformers?**

El modelo de transformers original está compuesto por un encoder y un decoder; cada uno es un stack (pila) de lo que podemos llamar bloques de transformers. Esta arquitectura era apropiada para tareas como la traducción automática, un problema en el que las arquitecturas encoder-decoder han tenido éxito en el pasado.

<center><img src="http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png" /></center>

Gran parte del trabajo de investigación posterior vio a la arquitectura deshacerse o bien del encoder o decoder, y usar solo un stack de bloques de transformers, apilándo tantos como fuese posible y alimentándolos con cantidades masivas de texto de entrenamiento y un enorme poder de computo, sin dejar de lado la basta inversión monetaria.

<center><img src="http://jalammar.github.io/images/gpt2/gpt-2-transformer-xl-bert-3.png" /></center>

El modelo GPT-2 es construido usando bloques de decoders. Al igual que los modelos de lenguaje tradicionales, genera un token a la vez.

<center><img src="https://miro.medium.com/max/700/1*C2pZ2jmq3XDNyzpHt6pSWQ.png" /></center>

La forma en que estos modelos realmente funcionan es que después de que se produce cada token, ese token se agrega a la secuencia de entradas. Esa nueva secuencia se convierte en la entrada al modelo en su siguiente paso. Esta idea es llamada "autoregresión" y es una de las ideas que hizo que las RNN fueran tan efectivas.

## **El Decoder-Only Block**

El modelo GPT-2 de OpenAI usa estos decoder-only blocks.


<center><img src="https://miro.medium.com/max/700/1*Z-P4_8w9wVhIfgYz32NSoQ.png" /></center>

Es importante que la distinción entre self-attention (usado por modelos como BERT) y masked self-attention (la que usa GPT-2) sea clara. Un bloque con una capa self-attention permite a las entradas intectactuar consigo mismas para aprender dependencias entre ellas y usar esa información para capturar la estructura interna de una frase. Por otra parte, la masked self-attention bloquea la información de los tokens que están a la derecha de la posición que está siendo calculada.

<center><img src="http://jalammar.github.io/images/gpt2/self-attention-and-masked-self-attention.png" /></center>

### **Descripción breve del modelo a utilizar:**

**English pre-trained GPT-2 small**
*  12 capas, 768-ocultas
*  124M de parámetros
*  Tiempo de descarga: aproximadamente 10 minutos


**English pre-trained Byte-level BPE tokenizer**
*  Byte-level BPE
*  vocabulario de 50.257 tokens

## **¿Qué es un tokenizer?**

Tokenizar consiste esencialmente en dividir una frase, oración, párrafo o un documento de texto en unidades más pequeñas, como palabras o términos individuales (caracteres). Cada una de estas unidades más pequeñas se llama token.

Para conocer más sobre el **BPE tokenizer**, véase: https://huggingface.co/transformers/master/tokenizer_summary.html

# **HUGGING FACE - Librería Transformers**



---



<center><img src="https://raw.githubusercontent.com/huggingface/transformers/master/docs/source/imgs/transformers_logo_name.png" /></center>

La librería **Transformers** provee el estado del arte en arquitecturas de Machine Learning como BERT, **GPT-2**, RoBERTa, XLM, DistilBert, XLNet, T5, para Natural Language Understanding (NLU), y Natural Language Generation (NLG). Provee cientos de modelos pre-entrenados en más de 100 idiomas diferentes siendo profundamente interoperables entre PyTorch y TensorFlow 2.0. Además, permite a los desarrolladores hacer **fine-tune** a dichos modelos para diferentes tareas de **NLP** como clasificación de texto, análisis de sentimientos o generación de texto.

# **Implementación**

Para la implementación, haremos **fine-tune** a la Spanish GPT-2 de los modelos pre-entrenados de Huggingface (esta implementación es a su vez producto de un proceso de fine-tune de la GPT-2 small original en Inglés). Respecto a los datos, usamos **web scrapping** para obtener tweets relacionados con la vacuna del covid-19 con un enfoque Hispanohablante (énfasis en Colombia). Llegamos a juntar alrededor de 17120 tweets.

Usaremos estos tweets para hacer fine-tune a nuestro modelo GPT-2 que nos permita generar tweets que, en principio, hablen positiviamente o fomenten el uso de la vacuna.

Implementación en Español de la GPT-2 Small: https://huggingface.co/datificate/gpt2-small-spanish 

### **Pasos para llevar a cabo la implementacón:**

* Cargar el dataset (datos recopilados por nosotros a través de **web scrapping**).
* Preparar el dataset y construir un **TextDataset**.
* Inicializar el **Trainer** con sus **TrainingArguments** y el modelo **GPT-2**.
* Entrenar y guardar el modelo.
* Testear el modelo.




---



In [ ]:
#@title **Importación de Librerias**
import tweepy as tw
import pandas as pd

In [ ]:
!pip install transformers==4.2.2

     |████████████████████████████████| 1.8MB 8.2MB/s 
     |████████████████████████████████| 2.9MB 45.4MB/s 
     |████████████████████████████████| 890kB 52.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=6060451a19deb7b89ceae738907603db7a3d6fd605e97f77f5844e33d7702cdb
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!nvidia-smi

Mon Mar  8 21:45:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Lectura y Procesamiento del Dataset**

Para la recolección de los datos, como se mencionó anteriormente, se realizó un proceso de web scrapping, del cual se extrajeron alrededor de 17000 tweets los cuales se cargan a continuación:

In [ ]:
data1 = pd.read_csv('../data/data_generator/tweets_covid_2241-2020-012-021.csv')
data2 = pd.read_csv('../data/data_generator/tweets_covid_5236-2021-03-04.csv')
data3 = pd.read_csv('../data/data_generator/tweets_covid_1083-2020-09-017.csv')
data4 = pd.read_csv('../data/data_classifier/tweets/tweets_covid_positivo5054.csv')
data5 = pd.read_csv('../data/data_generator/tweets_gobierno_clean.csv')

In [ ]:
tweets = pd.concat([data1, data2, data3, data4, data5])
tweets.reset_index(drop=True, inplace=True)

In [ ]:
tweets

,Nombre de usuario,Usuario,Fecha,Texto,Unnamed: 0
0,Carlos Arturo Ospina Vanegas,@CarlosA99231303,2020-12-21T23:54:40.000Z,Si nosotros compramos la vacuna directamente n...,NaN
1,Anvi,@IvannRodriguezn,2020-12-21T23:50:42.000Z,Colombia no puede negarle la vacuna a Venezola...,NaN
2,Carlos Daniel Galvis,@carlosdgalvis,2020-12-21T23:42:45.000Z,Después de escuchar al señor Presidente de la ...,NaN
3,Francesco Manetto,@fmanetto,2020-12-21T23:29:04.000Z,"Decenas, probablemente cientos de miles de per...",NaN
4,Isabel III,@IssaCgv,2020-12-21T23:26:22.000Z,Acabo de ver lo de Colombia con la exclusión d...,NaN
...,...,...,...,...,...
17115,SecretaríaSaludCesar,@CesarSecSalud,2020-03-09,"Hoy, @CesarSecSalud en conjunto con @SecSaludV...",981.0
17116,SecretaríaSaludCesar,@CesarSecSalud,2020-03-09,Descarga CoronApp y despeja todas tus dudas so...,982.0
17117,SecretaríaSaludCesar,@CesarSecSalud,2020-03-09,En la mesa sobre#Coronavirus #COVID19 particip...,980.0
17118,SecretaríaSaludCesar,@CesarSecSalud,2020-03-07,Seguimos avanzando en acciones contra el #Deng...,983.0


In [ ]:
tweets.dropna(inplace=True)
tweets.drop_duplicates(inplace=True)
tweets.reset_index(drop=True, inplace=True)

In [ ]:
tweets

,Nombre de usuario,Usuario,Fecha,Texto,Unnamed: 0
0,Carlos Arturo Ospina Vanegas,@CarlosA99231303,2020-12-21T23:54:40.000Z,Si nosotros compramos la vacuna directamente n...,NaN
1,Anvi,@IvannRodriguezn,2020-12-21T23:50:42.000Z,Colombia no puede negarle la vacuna a Venezola...,NaN
2,Carlos Daniel Galvis,@carlosdgalvis,2020-12-21T23:42:45.000Z,Después de escuchar al señor Presidente de la ...,NaN
3,Francesco Manetto,@fmanetto,2020-12-21T23:29:04.000Z,"Decenas, probablemente cientos de miles de per...",NaN
4,Isabel III,@IssaCgv,2020-12-21T23:26:22.000Z,Acabo de ver lo de Colombia con la exclusión d...,NaN
...,...,...,...,...,...
17115,SecretaríaSaludCesar,@CesarSecSalud,2020-03-09,"Hoy, @CesarSecSalud en conjunto con @SecSaludV...",981.0
17116,SecretaríaSaludCesar,@CesarSecSalud,2020-03-09,Descarga CoronApp y despeja todas tus dudas so...,982.0
17117,SecretaríaSaludCesar,@CesarSecSalud,2020-03-09,En la mesa sobre#Coronavirus #COVID19 particip...,980.0
17118,SecretaríaSaludCesar,@CesarSecSalud,2020-03-07,Seguimos avanzando en acciones contra el #Deng...,983.0


# **Creación de un dataset de texto**

El siguiente paso es extraer el campo 'texto' de todos los tweets y construir un **TextDataset**. El **TextDataset** es una implementación personalizada de la clase **Dataset** de Pytorch implementada por la biblioteca de transformers. Este objeto TextDataset, como todo en HuggingFace, facilita el proceso de creación de un dataset apto para entrenar y testear nuestro modelo.

La información para train y test se guardará en 'train_dataset.txt' y 'test_dataset.txt' respectivamente.


In [ ]:
import re
import json
from sklearn.model_selection import train_test_split


# with open('recipes.json') as f:
#     data = json.load(f)

def build_text_files(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    summaries = df['Texto'].tolist()
    for texts in summaries:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + " "
    f.write(data)

train, test = train_test_split(tweets, test_size=0.15) 


build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))


Train dataset length: 14552
Test dataset length: 2568


# **Cargar Tokenizers**

El siguiente paso es descargar el tokenizer. Usaremos el tokenizer del modelo en Español gpt-2 alojado en Huggingface.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Ahora podemos construir nuestro TextDataset. Para ello, creamos una instancia de TextDataset a la cual le suministramos el tokenizer y las rutas a nuestros datasets. Además, creamos un data_collator, el cual se utiliza en el entrenamiento para formar un batch a partir de nuestro dataset.


In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


# **Definir Parámetros de Entrenamiento**

La clase **Trainer** proporciona una API para un entrenamiento con todas las funciones. Se utiliza en la mayoría de los scripts de ejemplo de Huggingface. Antes de que podamos crear una instancia de Trainer, necesitamos descargar el modelo GPT-2 y crear un objeto **TrainingArguments**. Los TrainingArguments se utilizan para definir los hiperparámetros que usamos en el proceso de entrenamiento, como learning_rate, número de époocas, train_batch_size, etc. 

A continuación se puede encontrar la documentación de TrainingArguments: https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("datificate/gpt2-small-spanish")


training_args = TrainingArguments(
    output_dir="./gpt2-tuits+gobierno_50", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=50, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=4500, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


En este caso, he decidido entrenar con 50 épocas, un batch size de entrenamiento de 32, un batch size de evaluation de 64. El optimizador es el optimizador por defecto, en este caso el Adam, y de la misma manera, el learning rate se dejó por defecto, el cual es 5e-5.

# **Entrenar el modelo**

In [ ]:
trainer.train()

Step,Training Loss
500,4.504900
1000,3.710300
1500,3.347000
2000,3.087100
2500,2.880500
3000,2.709600
3500,2.566900


Step,Training Loss
500,4.504900
1000,3.710300
1500,3.347000
2000,3.087100
2500,2.880500
3000,2.709600
3500,2.566900
4000,2.452000
4500,2.364200
5000,2.293400


TrainOutput(global_step=6000, training_loss=2.865012268066406, metrics={'train_runtime': 7330.4464, 'train_samples_per_second': 0.819, 'total_flos': 18296832953548800, 'epoch': 50.0})

Podemos observar como el entrenamiento con un dataset bastante pequeño como el nuestro, para 50 épocas, tardó alrededor de 2 horas en completarse. Ahora, guardemos el modelo que acabamos de entrenar.

In [ ]:
trainer.save_model()

# **Testear el modelo - Generar texto**

Para probar el modelo, usaremos otra objeto destacado de la biblioteca de transformers llamad **pipeline**. Pipelines son objetos que ofrecen una API simple dedicada a varias tareas, tales como generación de texto, entre otras.

In [ ]:
from transformers import pipeline

generador_tweets = pipeline('text-generation',model='./gpt2-gobierno', tokenizer='datificate/gpt2-small-spanish',config={'max_length':800})


Una vez instanciado el pipeline, podemos generar tweets introduciendo un texto semilla, tal que:

In [ ]:
generador_tweets('El covid')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'El covid-19   #COVID19   #vacunassalvadoras Colombia sin vacuna!! El país en la peor crisis del mundo, donde las muertes y la falta de insumos hacen de los líderes más esenciales. La solución'

Como conclusión, podriamos decir que hemos hecho un proceso exitoso de fine-tune para la generación de tweets relacionados con la vacuna del covid19. Sin embargo, los resultados pueden no ser los mejores, lo cual es una consecuencia directa del tamaño insuficiente del dataset. Se podría intentar ajustar mejor los paramétros de entrenamiento o aumentar las épocas, aunque esto último, podría conducir a overfitting.



# **Implementación de un bot en Twitter** (sólo para usos educativos)

<img src="https://i.ibb.co/qprpK8v/uisbotxd.png" />

In [ ]:
access_key = '1366595402603126784-C0khbB5IuWkO65Z5YndOG48QTR8jVu'   
access_secret = 'qQNLpD7EZlqgwJXK6WT1hvxaPFUhitPO0Ei3nJn5RDUdX'
consumer_key = 'lJS4GDO5S8IQrkuelETTlAxpx'  
consumer_secret = 'f7uLovJsAmOV5UAXZ2JJABFRXcjKl3tpmsGBRfRbSGWWDWsY4k'

In [ ]:
#@title **Realizar autenticación con la API de Twitter**
import tweepy

# Authenticate to Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

# Create API object
api = tweepy.API(auth, wait_on_rate_limit=True,
    wait_on_rate_limit_notify=True)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [ ]:
#@title **Función para publicar tweets**
import time
semillas = ['Me parece', 'En el día de hoy, ', 'El covid', 'La vacuna', 'En Colombia']
# Create tweets
def publicarTweets(num_tweets, int_t):
  for i in range(num_tweets):
    api.update_status(generador_tweets(semillas[i])[0]['generated_text'])
    time.sleep(int_t)

In [ ]:
numTweets = 5 # Número de tweets a publicar.
t = 60 # Número de segundos a esperar hasta publicar otro tweet.
publicarTweets(numTweets, t)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


TweepError: ignored